In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets
# 모델 요약
from torchvision import models, utils
from torchsummary import summary

import time

In [2]:
## 네트워크를 저장하거나 불러오는 함수 작성하기
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               '%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()

    dict_model = torch.load('%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim

In [3]:
# 학습률, batch, epoch
lr = 1e-3
batch_size = 64
num_epoch = 10

# 학습된 network가 저장될 디렉터리
ckpt_dir = 'D:/torchtorch/checkpoint8'
# 텐서보드 파일이 저장될 디렉터리
log_dir = 'D:/torchtorch/log8'
# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# 기존 ImageNet 학습한 resnet18 모델 불러오기 
Model_RESNET18 = models.resnet18(pretrained=True)
# ResNet fc layer의 in_features
Num_Features = Model_RESNET18.fc.in_features
# 10개의 클래스만 분류하면 되므로 마지막 fc layer만 변경(fine tuning)
Model_RESNET18.fc = nn.Linear(Num_Features, 10).cuda()
# GPU 할당
Model_RESNET18 = Model_RESNET18.to(device)

In [5]:
Model_RESNET18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
for name, param in Model_RESNET18.named_parameters():
    if param.requires_grad == True:
        print (name, param.data)

conv1.weight tensor([[[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  ...,  5.6615e-02,
            1.7083e-02, -1.2694e-02],
          [ 1.1083e-02,  9.5276e-03, -1.0993e-01,  ..., -2.7124e-01,
           -1.2907e-01,  3.7424e-03],
          [-6.9434e-03,  5.9089e-02,  2.9548e-01,  ...,  5.1972e-01,
            2.5632e-01,  6.3573e-02],
          ...,
          [-2.7535e-02,  1.6045e-02,  7.2595e-02,  ..., -3.3285e-01,
           -4.2058e-01, -2.5781e-01],
          [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  ...,  4.1384e-01,
            3.9359e-01,  1.6606e-01],
          [-1.3736e-02, -3.6746e-03, -2.4084e-02,  ..., -1.5070e-01,
           -8.2230e-02, -5.7828e-03]],

         [[-1.1397e-02, -2.6619e-02, -3.4641e-02,  ...,  3.2521e-02,
            6.6221e-04, -2.5743e-02],
          [ 4.5687e-02,  3.3603e-02, -1.0453e-01,  ..., -3.1253e-01,
           -1.6051e-01, -1.2826e-03],
          [-8.3730e-04,  9.8420e-02,  4.0210e-01,  ...,  7.0789e-01,
            3.6887e-01,  1.2455e-01],
       

          [ 1.9402e-02,  2.8854e-02, -9.6113e-02]]]], device='cuda:0')
layer1.1.bn1.weight tensor([0.3910, 0.4375, 0.3746, 0.3990, 0.3404, 0.3503, 0.2618, 0.2707, 0.2865,
        0.4308, 0.1895, 0.3041, 0.3837, 0.2944, 0.2105, 0.3304, 0.2943, 0.2887,
        0.2060, 0.4627, 0.2335, 0.1831, 0.4489, 0.2830, 0.3389, 0.2997, 0.3503,
        0.2735, 0.3908, 0.2817, 0.2636, 0.4462, 0.3282, 0.3776, 0.4471, 0.3878,
        0.2516, 0.3172, 0.3661, 0.3166, 0.3818, 0.3128, 0.2274, 0.3627, 0.2902,
        0.2381, 0.2988, 0.2469, 0.3840, 0.2886, 0.3197, 0.2879, 0.3218, 0.4559,
        0.3500, 0.2420, 0.3396, 0.3519, 0.3839, 0.3806, 0.4039, 0.2826, 0.4594,
        0.3342], device='cuda:0')
layer1.1.bn1.bias tensor([-0.0997, -0.4755, -0.0474, -0.2698, -0.0834, -0.0072,  0.0474,  0.1022,
        -0.0170, -0.1471,  0.2307,  0.1447, -0.1775,  0.0273,  0.1559, -0.1836,
         0.1238, -0.1522,  0.0554, -0.2881, -0.2606,  0.2316, -0.3242, -0.0219,
        -0.2645,  0.0576, -0.2465,  0.0481, -0.3530,  0.0

       device='cuda:0')
layer2.0.downsample.1.bias tensor([ 0.0246,  0.0593,  0.1347, -0.1089, -0.0470, -0.1359, -0.0550,  0.0509,
        -0.0613,  0.0916,  0.0031, -0.0274, -0.0539,  0.0177,  0.0432,  0.0074,
         0.0548, -0.0321, -0.0224,  0.0142, -0.2150, -0.1160,  0.0486, -0.1141,
         0.1066,  0.0355,  0.0140,  0.0177,  0.0781,  0.1331,  0.0139,  0.0447,
         0.1063,  0.0528, -0.0539, -0.1160,  0.1055, -0.1591,  0.0100,  0.1197,
         0.0170,  0.0929, -0.0675,  0.0987,  0.1034,  0.0501,  0.0297,  0.0281,
        -0.0075, -0.0577, -0.0144, -0.1640,  0.1255,  0.0817,  0.0635,  0.0936,
         0.0213,  0.0486, -0.1174,  0.0237, -0.2177,  0.0099, -0.1883,  0.0467,
        -0.0829,  0.0585, -0.0306,  0.0509,  0.0541, -0.1671,  0.0115, -0.0302,
        -0.1393,  0.0115,  0.0428,  0.1189, -0.1289,  0.0479,  0.0474, -0.0625,
         0.0009, -0.0144,  0.0909,  0.1342, -0.0338,  0.0560,  0.0848, -0.0467,
         0.0228, -0.0097,  0.1360, -0.2625,  0.0088, -0.0553,  0.0383

layer3.0.bn1.bias tensor([-0.0915,  0.0189, -0.1235, -0.0613, -0.1003, -0.1306, -0.1473, -0.1079,
        -0.2438, -0.1113,  0.1361, -0.1477,  0.0387, -0.0907,  0.0352, -0.1851,
        -0.1319, -0.1746, -0.0815, -0.1004, -0.3394, -0.1712, -0.0807, -0.1228,
        -0.2263, -0.1503, -0.2314, -0.2327, -0.0854, -0.0802, -0.0716, -0.0839,
        -0.0592,  0.0358, -0.0322, -0.2197,  0.0027, -0.1471, -0.0264, -0.1886,
        -0.2417, -0.1494, -0.1904, -0.1089, -0.2657, -0.1362, -0.0487, -0.1340,
        -0.0930, -0.0064, -0.1721, -0.1476, -0.1714,  0.0336, -0.1011, -0.1761,
        -0.1184, -0.0482, -0.3260, -0.1555, -0.0169, -0.2373, -0.1015, -0.1051,
        -0.2738, -0.1917, -0.0503, -0.1098,  0.1484, -0.2282, -0.0700, -0.1427,
        -0.1417, -0.3096, -0.2043,  0.0269, -0.0779, -0.0842, -0.0464, -0.1429,
        -0.3917,  0.0257, -0.1779, -0.0993, -0.0507, -0.2222, -0.0951, -0.0861,
        -0.0743, -0.1666, -0.2054, -0.1782, -0.1150, -0.2525, -0.0694, -0.0536,
        -0.0499, -0.03

       device='cuda:0')
layer3.1.conv2.weight tensor([[[[-4.2568e-02, -2.6148e-02, -2.2019e-02],
          [-1.7334e-02, -7.5950e-03, -7.2384e-03],
          [-1.7876e-03,  2.3800e-02,  1.4873e-02]],

         [[-2.8277e-03, -5.0644e-03, -4.9442e-03],
          [ 1.2117e-03,  1.4908e-02,  1.6013e-02],
          [ 1.4391e-02,  3.3109e-02,  5.0061e-02]],

         [[-3.4891e-03, -4.4437e-03,  2.6589e-03],
          [ 1.5105e-02,  2.6303e-02,  2.6802e-02],
          [ 3.9232e-02,  5.0057e-02,  4.6637e-02]],

         ...,

         [[ 2.2877e-02,  1.5454e-02, -2.4483e-02],
          [ 3.1145e-02,  3.4944e-02,  1.3296e-02],
          [-1.7674e-04,  7.3297e-03, -5.7174e-03]],

         [[-2.1781e-02, -3.7379e-02, -1.3382e-02],
          [ 1.8976e-02,  1.4155e-02, -6.5395e-03],
          [ 2.6831e-02,  3.6354e-02,  1.1450e-02]],

         [[ 3.1603e-02,  3.3933e-02,  3.1575e-02],
          [-1.0098e-02, -1.2657e-02,  1.1674e-02],
          [ 1.0325e-02,  7.9424e-05,  1.5911e-02]]],


       

layer4.0.conv2.weight tensor([[[[ 1.6218e-04, -1.4720e-02, -1.7000e-02],
          [-1.2850e-02, -3.3085e-02, -3.6656e-02],
          [ 2.7812e-02,  1.7691e-02, -1.8369e-02]],

         [[ 1.0528e-02,  3.1379e-02,  2.4801e-02],
          [-1.2698e-02, -2.9453e-02, -1.1834e-02],
          [-9.4094e-03, -8.9462e-03, -3.1349e-02]],

         [[-7.8447e-03, -2.9256e-02,  5.3590e-03],
          [-1.3791e-02, -1.1116e-02,  5.0388e-03],
          [-2.4919e-03,  7.3514e-03,  5.4013e-03]],

         ...,

         [[-1.0276e-03, -1.0275e-02, -2.9986e-02],
          [-3.8465e-03,  1.9549e-03, -1.6291e-02],
          [-1.8100e-03,  8.3778e-03, -8.5481e-03]],

         [[-1.8196e-02, -1.3533e-02, -1.7457e-02],
          [ 2.2457e-02,  5.7402e-02,  1.9325e-02],
          [-2.4977e-02, -3.2113e-02, -8.1780e-03]],

         [[ 3.6550e-03,  4.9358e-03, -5.7597e-03],
          [-1.6875e-02,  1.3999e-04,  3.7629e-04],
          [-2.6272e-03,  1.0947e-03,  1.1145e-03]]],


        [[[ 1.4018e-02,  3.9198

layer4.1.bn1.weight tensor([0.2587, 0.3073, 0.2595, 0.3223, 0.2662, 0.2652, 0.2575, 0.2660, 0.2766,
        0.2414, 0.3045, 0.2853, 0.2821, 0.2880, 0.3094, 0.3444, 0.3155, 0.4129,
        0.2110, 0.2903, 0.2496, 0.2601, 0.2967, 0.3033, 0.4152, 0.2719, 0.3661,
        0.3251, 0.3898, 0.3346, 0.2753, 0.2712, 0.2414, 0.3351, 0.3394, 0.3167,
        0.3360, 0.2666, 0.2109, 0.2705, 0.2587, 0.3070, 0.2720, 0.2316, 0.2885,
        0.2884, 0.2955, 0.3057, 0.3043, 0.2596, 0.2673, 0.1929, 0.3136, 0.3593,
        0.2622, 0.2931, 0.3295, 0.2514, 0.3208, 0.2798, 0.3259, 0.2939, 0.2390,
        0.3105, 0.3471, 0.2812, 0.2148, 0.2997, 0.3061, 0.2740, 0.2791, 0.3790,
        0.3592, 0.3247, 0.2995, 0.2735, 0.3356, 0.2703, 0.3255, 0.3127, 0.2783,
        0.2702, 0.3900, 0.2942, 0.2899, 0.3461, 0.3432, 0.4685, 0.2634, 0.2553,
        0.3019, 0.3961, 0.2742, 0.2995, 0.3858, 0.2785, 0.3212, 0.3109, 0.3642,
        0.2193, 0.2643, 0.2333, 0.3151, 0.3102, 0.2936, 0.2374, 0.2419, 0.2976,
        0.3335, 0.26

In [7]:
## 모델 요약
print(summary(Model_RESNET18, (3,244,244)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]          36,864
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
       BasicBlock-11           [-1, 64, 61, 61]               0
           Conv2d-12           [-1, 64, 61, 61]          36,864
      BatchNorm2d-13           [-1, 64, 61, 61]             128
             ReLU-14           [-1, 64,

In [8]:
from torch.optim import lr_scheduler
## 필요한 손실함수 구현하기

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(Model_RESNET18.parameters(), lr=lr)

# 이전 learning rate 변경폭에 따라 학습률 재조정 시켜줌
# !! Train에서만 사용함
# mode='max': accuracy, 'min' : loss 탐색
scheduler = lr_scheduler.ReduceLROnPlateau(optim, patience=5, mode='min', factor = 0.5, verbose=1, threshold=0.003)

## early stopping condition 정의
n_epochs_stop = 10
epochs_no_improve = 0
early_stop = False
min_val_loss = np.Inf

writer = SummaryWriter(log_dir=log_dir)

In [9]:
## CIFAR10 데이터 불러오기
## 이미지 전처리 과정
# 1. 이미지 텐서 변환 및 정규화 등 수행(.Compose) 
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,), std=(0.5,)),
                                # 이미지 resize, resize하면서 생기는 pixel은 BILINEAR interpolation 수행
                                transforms.Resize(224, interpolation=2)])

# 2. 데이터셋 불러오기
dataset = datasets.CIFAR10(download=True, root='./.data/', train=True, transform=transform)

# 3. 데이터셋 나누기
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# 4. 1배치에 들어갈 데이터 
# 1epoch = 50000개 = 64batch
# 1batch = 50000/64(batch_size) = 938
num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

Files already downloaded and verified


In [10]:
print(num_data)
print(num_batch)

50000
782.0


In [11]:
## 데이터셋 시각화하기
# 배치 1개의 image와 label 들을 확인해보기(64개 이미지 존재)
dataiter = iter(loader)
images, labels = next(dataiter)
# 시각화 하기
img = utils.make_grid(images, padding=0)
# 원래 이미지 -> 64개(1epoch, channel, height, width)
print(images.shape)
print(labels.shape)
print(labels)
npimg = img.numpy()
plt.figure(figsize=(10,7))
plt.imshow(np.transpose(npimg, (1,2,0)))
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


torch.Size([64, 3, 224, 224])
torch.Size([64])
tensor([7, 3, 4, 8, 0, 2, 7, 0, 9, 0, 1, 2, 0, 1, 2, 4, 6, 3, 4, 3, 7, 3, 8, 8,
        9, 4, 9, 6, 7, 5, 3, 2, 5, 4, 1, 4, 2, 0, 6, 7, 3, 2, 6, 5, 2, 7, 2, 3,
        9, 2, 3, 5, 4, 5, 6, 8, 8, 1, 2, 8, 1, 7, 7, 6])


In [12]:
# 클래스 딕셔너리로 만들기
CLASSES = {
    0 : 'plane',
    1 : 'car',
    2 : 'bird',
    3 : 'cat',
    4 : 'deer',
    5 : 'dog',
    6 : 'frog',
    7 : 'horse',
    8 : 'ship',
    9 : 'truck'
}
# labels 는 0부터 9까지의 숫자로 이루어져 있다.
# 이를 CLASSES에 있는 index와 일치시킨다.
# print(labels)
# for label in labels :
#     index = label.item()
#     print(CLASSES[index])

In [13]:
## 트레이닝 시작하기
st_epoch = 0
# train 시작시간 
st_time = time.time()
 
for epoch in range(st_epoch + 1, num_epoch + 1):
    Model_RESNET18.train()
    
    ## early stopping condition 각 epoch마다 초기화해주기
    epochs_no_improve = 0
    early_stop = False
    min_val_loss = np.Inf
    
    loss_arr = []
    acc_arr = []

    for  batch, (input, label) in enumerate(loader, 1):
        
        input = input.to(device)
        # input shape = (batch, channel, height, width)
#         print(input.shape)
        
        label = label.to(device)
        # label shape = (batch)
#         print(label.shape)
        
        output = Model_RESNET18(input)
        # output shape = (batch, 10)
#         print(output.shape)
        
        # pred shape = (batch, 10)
        pred = fn_pred(output)
#         print(pred.shape)
        
        # 모든 model parameter의 gradient를 0으로 설정한다.(epoch마다 새롭게, loss 증가 방지)
        optim.zero_grad()

        # loss와 acc를 계산
        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)
    
        # loss와 chain rule를 통해 각 layer의 gradient를 계산한다.
        loss.backward()
        
        # parameter를 update한다.
        optim.step()

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))
        
        scheduler.step(np.mean(loss_arr))
        
        # loss가 향상되지 않았다면 count를 증가시킨다.
        if np.mean(loss_arr) < min_val_loss :
            epochs_no_improve = 0
            min_val_loss = np.mean(loss_arr)
        else :
            epochs_no_improve += 1
        # 만약 향상되지 않은 count가 지정한 횟수에 도달하면 빠져나간다.
        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!' )
            early_stop = True
            print("Stopped")
            break
        else:
            continue
    
    writer.add_scalar('loss', np.mean(loss_arr), epoch)
    writer.add_scalar('acc', np.mean(acc_arr), epoch)
    
    save(ckpt_dir = ckpt_dir, net=Model_RESNET18, optim=optim, epoch=epoch)
    
# train 종료 시간 
time_elapsed = time.time() - st_time
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
writer.close()

TRAIN: EPOCH 0001/0010 | BATCH 0001/0782 | LOSS: 2.4986 | ACC 0.0781
TRAIN: EPOCH 0001/0010 | BATCH 0002/0782 | LOSS: 2.0976 | ACC 0.2344
TRAIN: EPOCH 0001/0010 | BATCH 0003/0782 | LOSS: 1.9711 | ACC 0.2969
TRAIN: EPOCH 0001/0010 | BATCH 0004/0782 | LOSS: 1.7798 | ACC 0.3633
TRAIN: EPOCH 0001/0010 | BATCH 0005/0782 | LOSS: 1.6903 | ACC 0.3969
TRAIN: EPOCH 0001/0010 | BATCH 0006/0782 | LOSS: 1.6039 | ACC 0.4271
TRAIN: EPOCH 0001/0010 | BATCH 0007/0782 | LOSS: 1.5771 | ACC 0.4353
TRAIN: EPOCH 0001/0010 | BATCH 0008/0782 | LOSS: 1.5575 | ACC 0.4434
TRAIN: EPOCH 0001/0010 | BATCH 0009/0782 | LOSS: 1.5035 | ACC 0.4601
TRAIN: EPOCH 0001/0010 | BATCH 0010/0782 | LOSS: 1.4788 | ACC 0.4766
TRAIN: EPOCH 0001/0010 | BATCH 0011/0782 | LOSS: 1.4779 | ACC 0.4787
TRAIN: EPOCH 0001/0010 | BATCH 0012/0782 | LOSS: 1.4466 | ACC 0.4935
TRAIN: EPOCH 0001/0010 | BATCH 0013/0782 | LOSS: 1.4193 | ACC 0.5072
TRAIN: EPOCH 0001/0010 | BATCH 0014/0782 | LOSS: 1.3959 | ACC 0.5179
TRAIN: EPOCH 0001/0010 | BATCH 001

TRAIN: EPOCH 0001/0010 | BATCH 0120/0782 | LOSS: 0.8808 | ACC 0.6964
TRAIN: EPOCH 0001/0010 | BATCH 0121/0782 | LOSS: 0.8771 | ACC 0.6978
TRAIN: EPOCH 0001/0010 | BATCH 0122/0782 | LOSS: 0.8762 | ACC 0.6988
TRAIN: EPOCH 0001/0010 | BATCH 0123/0782 | LOSS: 0.8746 | ACC 0.6993
TRAIN: EPOCH 0001/0010 | BATCH 0124/0782 | LOSS: 0.8720 | ACC 0.7002
TRAIN: EPOCH 0001/0010 | BATCH 0125/0782 | LOSS: 0.8697 | ACC 0.7013
TRAIN: EPOCH 0001/0010 | BATCH 0126/0782 | LOSS: 0.8669 | ACC 0.7025
TRAIN: EPOCH 0001/0010 | BATCH 0127/0782 | LOSS: 0.8648 | ACC 0.7034
TRAIN: EPOCH 0001/0010 | BATCH 0128/0782 | LOSS: 0.8631 | ACC 0.7039
TRAIN: EPOCH 0001/0010 | BATCH 0129/0782 | LOSS: 0.8610 | ACC 0.7046
TRAIN: EPOCH 0001/0010 | BATCH 0130/0782 | LOSS: 0.8594 | ACC 0.7050
TRAIN: EPOCH 0001/0010 | BATCH 0131/0782 | LOSS: 0.8573 | ACC 0.7057
TRAIN: EPOCH 0001/0010 | BATCH 0132/0782 | LOSS: 0.8545 | ACC 0.7069
TRAIN: EPOCH 0001/0010 | BATCH 0133/0782 | LOSS: 0.8527 | ACC 0.7072
TRAIN: EPOCH 0001/0010 | BATCH 013

TRAIN: EPOCH 0001/0010 | BATCH 0238/0782 | LOSS: 0.7253 | ACC 0.7521
TRAIN: EPOCH 0001/0010 | BATCH 0239/0782 | LOSS: 0.7241 | ACC 0.7525
TRAIN: EPOCH 0001/0010 | BATCH 0240/0782 | LOSS: 0.7228 | ACC 0.7529
TRAIN: EPOCH 0001/0010 | BATCH 0241/0782 | LOSS: 0.7218 | ACC 0.7534
TRAIN: EPOCH 0001/0010 | BATCH 0242/0782 | LOSS: 0.7204 | ACC 0.7539
TRAIN: EPOCH 0001/0010 | BATCH 0243/0782 | LOSS: 0.7192 | ACC 0.7541
TRAIN: EPOCH 0001/0010 | BATCH 0244/0782 | LOSS: 0.7181 | ACC 0.7544
TRAIN: EPOCH 0001/0010 | BATCH 0245/0782 | LOSS: 0.7175 | ACC 0.7546
TRAIN: EPOCH 0001/0010 | BATCH 0246/0782 | LOSS: 0.7158 | ACC 0.7551
TRAIN: EPOCH 0001/0010 | BATCH 0247/0782 | LOSS: 0.7149 | ACC 0.7554
TRAIN: EPOCH 0001/0010 | BATCH 0248/0782 | LOSS: 0.7139 | ACC 0.7557
TRAIN: EPOCH 0001/0010 | BATCH 0249/0782 | LOSS: 0.7124 | ACC 0.7561
TRAIN: EPOCH 0001/0010 | BATCH 0250/0782 | LOSS: 0.7111 | ACC 0.7566
TRAIN: EPOCH 0001/0010 | BATCH 0251/0782 | LOSS: 0.7097 | ACC 0.7570
TRAIN: EPOCH 0001/0010 | BATCH 025

TRAIN: EPOCH 0001/0010 | BATCH 0357/0782 | LOSS: 0.6199 | ACC 0.7868
TRAIN: EPOCH 0001/0010 | BATCH 0358/0782 | LOSS: 0.6190 | ACC 0.7871
TRAIN: EPOCH 0001/0010 | BATCH 0359/0782 | LOSS: 0.6189 | ACC 0.7873
TRAIN: EPOCH 0001/0010 | BATCH 0360/0782 | LOSS: 0.6180 | ACC 0.7876
TRAIN: EPOCH 0001/0010 | BATCH 0361/0782 | LOSS: 0.6170 | ACC 0.7880
TRAIN: EPOCH 0001/0010 | BATCH 0362/0782 | LOSS: 0.6161 | ACC 0.7884
TRAIN: EPOCH 0001/0010 | BATCH 0363/0782 | LOSS: 0.6159 | ACC 0.7885
TRAIN: EPOCH 0001/0010 | BATCH 0364/0782 | LOSS: 0.6157 | ACC 0.7885
TRAIN: EPOCH 0001/0010 | BATCH 0365/0782 | LOSS: 0.6152 | ACC 0.7887
TRAIN: EPOCH 0001/0010 | BATCH 0366/0782 | LOSS: 0.6142 | ACC 0.7889
TRAIN: EPOCH 0001/0010 | BATCH 0367/0782 | LOSS: 0.6135 | ACC 0.7892
TRAIN: EPOCH 0001/0010 | BATCH 0368/0782 | LOSS: 0.6129 | ACC 0.7895
TRAIN: EPOCH 0001/0010 | BATCH 0369/0782 | LOSS: 0.6123 | ACC 0.7897
TRAIN: EPOCH 0001/0010 | BATCH 0370/0782 | LOSS: 0.6111 | ACC 0.7902
TRAIN: EPOCH 0001/0010 | BATCH 037

TRAIN: EPOCH 0001/0010 | BATCH 0475/0782 | LOSS: 0.5550 | ACC 0.8103
TRAIN: EPOCH 0001/0010 | BATCH 0476/0782 | LOSS: 0.5547 | ACC 0.8105
TRAIN: EPOCH 0001/0010 | BATCH 0477/0782 | LOSS: 0.5543 | ACC 0.8106
TRAIN: EPOCH 0001/0010 | BATCH 0478/0782 | LOSS: 0.5535 | ACC 0.8109
TRAIN: EPOCH 0001/0010 | BATCH 0479/0782 | LOSS: 0.5531 | ACC 0.8110
TRAIN: EPOCH 0001/0010 | BATCH 0480/0782 | LOSS: 0.5523 | ACC 0.8112
TRAIN: EPOCH 0001/0010 | BATCH 0481/0782 | LOSS: 0.5515 | ACC 0.8115
TRAIN: EPOCH 0001/0010 | BATCH 0482/0782 | LOSS: 0.5511 | ACC 0.8117
TRAIN: EPOCH 0001/0010 | BATCH 0483/0782 | LOSS: 0.5506 | ACC 0.8119
TRAIN: EPOCH 0001/0010 | BATCH 0484/0782 | LOSS: 0.5502 | ACC 0.8120
TRAIN: EPOCH 0001/0010 | BATCH 0485/0782 | LOSS: 0.5496 | ACC 0.8122
TRAIN: EPOCH 0001/0010 | BATCH 0486/0782 | LOSS: 0.5493 | ACC 0.8123
TRAIN: EPOCH 0001/0010 | BATCH 0487/0782 | LOSS: 0.5487 | ACC 0.8125
TRAIN: EPOCH 0001/0010 | BATCH 0488/0782 | LOSS: 0.5482 | ACC 0.8127
TRAIN: EPOCH 0001/0010 | BATCH 048

TRAIN: EPOCH 0001/0010 | BATCH 0594/0782 | LOSS: 0.5031 | ACC 0.8280
TRAIN: EPOCH 0001/0010 | BATCH 0595/0782 | LOSS: 0.5029 | ACC 0.8281
TRAIN: EPOCH 0001/0010 | BATCH 0596/0782 | LOSS: 0.5025 | ACC 0.8282
TRAIN: EPOCH 0001/0010 | BATCH 0597/0782 | LOSS: 0.5021 | ACC 0.8284
TRAIN: EPOCH 0001/0010 | BATCH 0598/0782 | LOSS: 0.5015 | ACC 0.8286
TRAIN: EPOCH 0001/0010 | BATCH 0599/0782 | LOSS: 0.5011 | ACC 0.8287
TRAIN: EPOCH 0001/0010 | BATCH 0600/0782 | LOSS: 0.5008 | ACC 0.8287
TRAIN: EPOCH 0001/0010 | BATCH 0601/0782 | LOSS: 0.5005 | ACC 0.8289
TRAIN: EPOCH 0001/0010 | BATCH 0602/0782 | LOSS: 0.4999 | ACC 0.8290
TRAIN: EPOCH 0001/0010 | BATCH 0603/0782 | LOSS: 0.4997 | ACC 0.8291
TRAIN: EPOCH 0001/0010 | BATCH 0604/0782 | LOSS: 0.4992 | ACC 0.8293
TRAIN: EPOCH 0001/0010 | BATCH 0605/0782 | LOSS: 0.4990 | ACC 0.8294
TRAIN: EPOCH 0001/0010 | BATCH 0606/0782 | LOSS: 0.4988 | ACC 0.8294
TRAIN: EPOCH 0001/0010 | BATCH 0607/0782 | LOSS: 0.4984 | ACC 0.8296
TRAIN: EPOCH 0001/0010 | BATCH 060

TRAIN: EPOCH 0001/0010 | BATCH 0711/0782 | LOSS: 0.4660 | ACC 0.8410
TRAIN: EPOCH 0001/0010 | BATCH 0712/0782 | LOSS: 0.4656 | ACC 0.8411
TRAIN: EPOCH 0001/0010 | BATCH 0713/0782 | LOSS: 0.4657 | ACC 0.8411
TRAIN: EPOCH 0001/0010 | BATCH 0714/0782 | LOSS: 0.4655 | ACC 0.8411
TRAIN: EPOCH 0001/0010 | BATCH 0715/0782 | LOSS: 0.4653 | ACC 0.8412
TRAIN: EPOCH 0001/0010 | BATCH 0716/0782 | LOSS: 0.4650 | ACC 0.8413
TRAIN: EPOCH 0001/0010 | BATCH 0717/0782 | LOSS: 0.4647 | ACC 0.8414
Epoch   717: reducing learning rate of group 0 to 1.5625e-05.
TRAIN: EPOCH 0001/0010 | BATCH 0718/0782 | LOSS: 0.4643 | ACC 0.8416
TRAIN: EPOCH 0001/0010 | BATCH 0719/0782 | LOSS: 0.4641 | ACC 0.8416
TRAIN: EPOCH 0001/0010 | BATCH 0720/0782 | LOSS: 0.4636 | ACC 0.8418
TRAIN: EPOCH 0001/0010 | BATCH 0721/0782 | LOSS: 0.4635 | ACC 0.8418
TRAIN: EPOCH 0001/0010 | BATCH 0722/0782 | LOSS: 0.4630 | ACC 0.8420
TRAIN: EPOCH 0001/0010 | BATCH 0723/0782 | LOSS: 0.4627 | ACC 0.8421
TRAIN: EPOCH 0001/0010 | BATCH 0724/0782 

TRAIN: EPOCH 0004/0010 | BATCH 0008/0782 | LOSS: 0.1430 | ACC 0.9668
Epoch   825: reducing learning rate of group 0 to 4.8828e-07.
TRAIN: EPOCH 0004/0010 | BATCH 0009/0782 | LOSS: 0.1458 | ACC 0.9670
TRAIN: EPOCH 0004/0010 | BATCH 0010/0782 | LOSS: 0.1482 | ACC 0.9641
TRAIN: EPOCH 0004/0010 | BATCH 0011/0782 | LOSS: 0.1528 | ACC 0.9602
TRAIN: EPOCH 0004/0010 | BATCH 0012/0782 | LOSS: 0.1526 | ACC 0.9609
TRAIN: EPOCH 0004/0010 | BATCH 0013/0782 | LOSS: 0.1562 | ACC 0.9591
TRAIN: EPOCH 0004/0010 | BATCH 0014/0782 | LOSS: 0.1533 | ACC 0.9609
Epoch   831: reducing learning rate of group 0 to 2.4414e-07.
TRAIN: EPOCH 0004/0010 | BATCH 0015/0782 | LOSS: 0.1521 | ACC 0.9615
Early stopping!
Stopped
TRAIN: EPOCH 0005/0010 | BATCH 0001/0782 | LOSS: 0.2047 | ACC 0.9062
TRAIN: EPOCH 0005/0010 | BATCH 0002/0782 | LOSS: 0.1620 | ACC 0.9375
TRAIN: EPOCH 0005/0010 | BATCH 0003/0782 | LOSS: 0.1350 | ACC 0.9583
TRAIN: EPOCH 0005/0010 | BATCH 0004/0782 | LOSS: 0.1286 | ACC 0.9648
TRAIN: EPOCH 0005/0010 |